In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}



In [ ]:
full_list = []

for i in range(1,10):
  page_url = 'https://able2know.org/forum/hiv/'+ 'page-' + str(i)
  print(page_url)
  r = requests.get(page_url, headers = headers)
  soup = BeautifulSoup(r.text, 'html.parser')

  threads_on_page = soup.find_all('div',{'style':'overflow:hidden;'})#[0].text
#len(threads)
  print(len(threads_on_page))

  ##looping on threads
  for j in range(len(threads_on_page)):
    temp_list=[]
    thread_specifc_url= threads_on_page[j].find('a')['href']
    print(thread_specifc_url)
    r1 = requests.get(thread_specifc_url, headers = headers)
    soup1 = BeautifulSoup(r1.text, 'html.parser')

    #username
    username= soup1.find_all('div',{'class':'header'})[0].find('a').text
    print(username)
    #user_url
    user_url= soup1.find_all('div',{'class':'header'})[0].find('a')['href']
    print(user_url)

    r2 = requests.get(user_url, headers = headers)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    
    #user join date
    join_date= soup2.find(('div'),{'class':'smalltxt'}).text
    
    #userProfilepage
    profile_stats= soup2.find_all('div',{'style':'height:357px;'})[0].find_all('li')

    

    #profile stats- basic
    total_answered_questions= profile_stats[0].text.split(':')[1]
    total_posts= profile_stats[1].text.split(':')[1]
    location= profile_stats[2].text.split(':')[1]
    occupation = (profile_stats[3].text.split(':'))[1]

    #profile stats- additional
    profile_follow_stats= soup2.find_all('div')

    profile_data= profile_follow_stats[16].text

    following_follower_list=profile_follow_stats[17].text.replace('\n','').replace('\t','').replace('Following ','').replace('Followers ','').split('\r')
    #following_follower_list

    try:
      following= following_follower_list[1]
    except Exception as e:
      following= "Is following some people. "+ "URL= "+ user_url

    try:
      followers= following_follower_list[2]
    except Exception as e:
      followers= "Is followed by some people. "+ "URL= "+ user_url
    #tags-list
    my_tags=soup2.find_all('div',{'class':'wrapper mainWrapper'})[0].find_all('a')#.text#['href']
    

    tags_list=[]
    for items in my_tags:
      fullstring = str(items)
      substring = "tags"
      if fullstring.find(substring) != -1:
        splitted_list= fullstring.split('>')
        #print(splitted_list)
        tags_list.append(splitted_list[1].replace('</a',''))

    #total_count_of_available_replies
    recent_posts= soup2.find_all('div',{'class':'title'})
    total_count_of_available_replies= len(recent_posts)
    print('total_count_of_available_replies', total_count_of_available_replies)
    
    
    
        
    temp_list=[username, user_url, join_date, tags_list, total_answered_questions, total_posts , location , occupation , profile_data, following, followers, total_count_of_available_replies]
    full_list.append(temp_list)
        
df = pd.DataFrame(full_list,columns=("Profile Name","Profile URL", "Join Date", "Groups tagged in","total_answered_questions","total_posts ", "Location","Occupation","Profile Data", "Following", "Followers", "total_available_responses"))
    

    

pd.set_option("max_colwidth",None)



In [4]:
df.to_csv('profile_extract_part1_SA_hiv.csv', index=False)

In [5]:
df.head()

,Profile Name,Profile URL,Join Date,Groups tagged in,total_answered_questions,total_posts,Location,Occupation,Profile Data,Following,Followers,total_available_responses
0,justaskin,https://able2know.org/user/justaskin/,"Member since August 26, 2020",[Hiv],0,3,,,No Profile,User has not yet added friends.,User does not yet have followers.,3
1,Khorospoos,https://able2know.org/user/khorospoos/,"Member since April 19, 2020","[Hiv, Drug Resistance]",0,1,,,No Profile,User has not yet added friends.,User does not yet have followers.,1
2,Feainn,https://able2know.org/user/feainn/,"Member since April 17, 2020","[Health, Hiv]",0,1,,,No Profile,User has not yet added friends.,User does not yet have followers.,1
3,Guy01,https://able2know.org/user/guy01/,"Member since November 12, 2019","[Semen, Vagina Fluid, Condom Lube, Slip, Sex, Hiv, Condom]",0,1,,,No Profile,User has not yet added friends.,User does not yet have followers.,1
4,Dda79,https://able2know.org/user/dda79/,"Member since July 5, 2019",[Hiv],0,1,,,No Profile,User has not yet added friends.,User does not yet have followers.,1


In [16]:
df.shape

(44, 12)

In [ ]:

full_final_list=[]
for index, row in df.iterrows():
    temp_list1 =[]

    if int(row['total_available_responses'])==0:
      
      post_url= 'NA'
      response_text= 'NA'
      response_time='NA'
      temp_list1=[row["Profile Name"], row["Profile URL"], row["Join Date"], row["Groups tagged in"], row["total_answered_questions"], row["total_posts"], row["Location"], row["Occupation"], row["Profile Data"], row["Following"], row["Followers"],row["total_available_responses"],reply_url, response_text, response_time]
      full_final_list.append(temp_list1)
    
    else:
      user_url= row["Profile URL"]
      
      r3 = requests.get(user_url, headers = headers)
      soup3 = BeautifulSoup(r3.text, 'html.parser')
      
      #total_count_of_available_replies
      recent_posts= soup3.find_all('div',{'class':'title'})
      total_count_of_available_replies= len(recent_posts)

      for k in range(len(recent_posts)):
        post_url= recent_posts[k].find('a')['href']
        print(post_url)
        r4 = requests.get(post_url, headers = headers)
        soup4 = BeautifulSoup(r4.text, 'html.parser')

        div_id= post_url.split('#')[1] + '-' + 'body' 
        response_text= soup4.find_all('div',{'id':div_id})[0].text.replace('\n','').replace('\t','')

        #response_time
        post_id= post_url.split('#')[1]

        try:
          response_time= soup4.find_all('div',{'id':post_id})[0].find('span', {'class': 'date smalltxt'}).text

        except Exception as e:
          response_time= 'NA'
        temp_list1= temp_list1=[row["Profile Name"], row["Profile URL"], row["Join Date"], row["Groups tagged in"], row["total_answered_questions"], row["total_posts "], row["Location"], row["Occupation"], row["Profile Data"], row["Following"], row["Followers"],row["total_available_responses"], post_url, response_text, response_time]
        full_final_list.append(temp_list1)



In [19]:
df_final = pd.DataFrame(full_final_list,columns=("Profile Name","Profile URL", "Join Date", "Groups tagged in","total_answered_questions","total_posts ", "Location","Occupation","Profile Data", "Following", "Followers", "total_available_responses", "specific_reply_url", "response_text","response_time"))

In [20]:
df_final.shape

(240, 15)

In [21]:
df_final.to_csv('profile_extract_SA_hiv.csv', index= False)

In [ ]:
#thread_specifc_url= threads[0].find('a')['href']

In [ ]:
# r1 = requests.get(thread_specifc_url, headers = headers)
# soup1 = BeautifulSoup(r1.text, 'html.parser')

In [ ]:
#username= soup1.find_all('div',{'class':'header'})[0].find('a').text#['href']
username

'justaskin'

In [ ]:
#user_url= soup1.find_all('div',{'class':'header'})[0].find('a')['href']

In [ ]:
#user_url

'https://able2know.org/user/justaskin/'

In [ ]:
# r2 = requests.get(user_url, headers = headers)
# soup2 = BeautifulSoup(r2.text, 'html.parser')

In [ ]:
#userProfilepage

#profile_stats= soup2.find_all('div',{'style':'height:357px;'})[0].find_all('li')#[0]#.find('a')['href']
#profile_stats

[<li><b>Answered Questions:</b> 0</li>,
 <li><b>Posts:</b> 3</li>,
 <li><b>Location:</b> </li>,
 <li><b>Occupation:</b> </li>]

In [ ]:

# total_answered_questions= profile_stats[0].text.split(':')[1]
# total_posts= profile_stats[1].text.split(':')[1]
# location= profile_stats[2].text.split(':')[1]
# occupation = (profile_stats[3].text.split(':'))[1]


In [ ]:
# total_answered_questions
# #total_posts
# #location
# #occupation

' 0'

In [ ]:
profile_follow_stats= soup2.find_all('div')#,{'class':'dividerText margin'})#[0]
len(profile_follow_stats)

50

In [ ]:
#profile_data= profile_follow_stats[16].text#.find('div')#.text
#profile_data



'No Profile'

In [ ]:
# following_follower_list=profile_follow_stats[17].text.replace('\n','').replace('\t','').replace('Following ','').replace('Followers ','').split('\r')
# following_follower_list

# following= following_follower_list[1]
# follower= following_follower_list[2]

In [ ]:
following

'User has not yet added friends.'

In [ ]:
# my_tags=soup2.find_all('div',{'class':'wrapper mainWrapper'})[0].find_all('a')#.text#['href']
# len(my_tags)
# my_tags[3].text

# tags_list=[]
# for items in my_tags:
#   fullstring = str(items)
#   substring = "tags"
#   if fullstring.find(substring) != -1:
#     splitted_list= fullstring.split('>')
#     print(splitted_list)
#     tags_list.append(splitted_list[1].replace('</a',''))



['<a href="https://able2know.org/user/justaskin/tags/hiv/"', 'Hiv</a', '']


In [ ]:
tags_list

['Hiv']

In [ ]:
# recent_posts= soup2.find_all('div',{'class':'title'})
# recent_posts

[<div class="title"><b><a href="https://able2know.org/topic/551459-1#post-7051767">Paranoia and protected sex</a></b></div>,
 <div class="title"><b><a href="https://able2know.org/topic/551459-1#post-7051753">Paranoia and protected sex</a></b></div>,
 <div class="title"><b><a href="https://able2know.org/topic/551459-1#post-7051732">Paranoia and protected sex</a></b></div>]

In [ ]:
recent_posts[0].find('a')['href'].split('#')

['https://able2know.org/topic/551459-1', 'post-7051767']

In [ ]:

for k in range(len(recent_posts)):
  post_url= recent_posts[k].find('a')['href']
  r3 = requests.get(post_url, headers = headers)
  soup3 = BeautifulSoup(r3.text, 'html.parser')

  div_id= post_url.split('#')[1] + '-' + 'body' 
  response_text= soup3.find_all('div',{'id':div_id})[0].text.replace('\n','').replace('\t','')

  #response_time
  post_id= post_url.split('#')[1]
  response_time= soup3.find_all('div',{'id':post_id})[0].find('span', {'class': 'date smalltxt'}).text
  print(response_time) 
  print(response_text)
  print("="*100)


							Wed 26 Aug, 2020 08:03 am
						
@PUNKEY,I never have sex without protection, in fact barely at all. No same sex encounters Im hetero. No anal never.No oral either just vaginal with con. This particular was one time encounter. She seems to follow safe sex rules requesting condom at all times. I was not even full length inside her at any times and that outer part briefly touched that 2mm skin damaged part on my finger.I could find her though and ask but my doc think Im overreacting. 

							Wed 26 Aug, 2020 07:31 am
						
@maxdancona,Hi,thanks for the thoughts.Actually I took the test today to be sure that prior to this intercourse I am clean. Surely will take another one in 2-3 months time just to evaluate this intercourse as well.Its just that I hate taking risks.There is PEP those are to be taken some time after problematic intercourse have a lookhttps://www.cdc.gov/hiv/basics/pep.htmlbut my doctor thinks I am unserious with this whole issue.

							Wed 26 Aug, 2020 06:13 